# Compute $\chi^2$ profile for J-factor at given ROI
## Takes as input jfactorgrid_gammaXX.dat (output from c++ code in Jfactor folder)

In [14]:
from __future__ import division
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: TkAgg


In [2]:
from matplotlib import rc
rc('font', family='times new roman', size=22)
rc('text', usetex=True)

### Load J-factor grid

In [116]:
# Path J-factor grid (jactorgrid_gammaXX.dat)
filepath = "./Jfactor/spherical_ROI/"
# Inner DM slope
gamma    = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
chi2     = []
logJ     = []
for g in gamma:
    print g
    data = np.genfromtxt(filepath + ("jfactorgrid_gamma%.1f.dat" %g), unpack=True)
    for i in range(len(data[0])):
        if data[1][i] != 0:
            chi2.append(data[0][i])
            logJ.append(np.log10(data[1][i]))
chi2 = np.array(chi2)
logJ = np.array(logJ)

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
1.1
1.2
1.3
1.4
1.5


### $\mathcal{J}$-factor vs. $\chi^2$

In [92]:
def bin_in_logJ(logJ, chi, nbins):
    logJ_edges = np.linspace(min(logJ), min(logJ)+8, nbins)
    logJbin = []
    chibin  = []
    for i in range(len(logJ_edges)-1):
        pos_bin = np.where((logJ >= logJ_edges[i]) & (logJ <= logJ_edges[i+1]))
        chibin.append(np.min(chi[pos_bin]))
        logJbin.append((logJ_edges[i+1] + logJ_edges[i])/2.)
    return np.array(logJbin), np.array(chibin)

In [117]:
logJbin, chi2bin = bin_in_logJ(logJ, chi2, 100)

In [118]:
# Plot chi2 grid
fig, ax = plt.subplots()
ax.plot(logJbin, chi2bin)
ax.set_yscale("log")

### Find $\mathcal{J}_{BF}$ (i.e. $\mathcal{J}$ with minimum $\chi^2$) and $\mathcal{J}_{min, max}$ (i.e. minimum and maximum $\mathcal{J}$ that satisfy $\chi^2 \leq \chi^2_{BF}+\Delta\chi^2$)

where $\Delta\chi^2$ is the 1, 2$\sigma$ $\chi^2$ corresponding to 1 or 2 d.o.f.

In [119]:
dof   = 1
sigma = 1
ci    = scipy.stats.chi2.cdf(sigma**2, 1)
chi2_1sigma = scipy.stats.chi2.ppf(ci, dof)
sigma = 2
ci    = scipy.stats.chi2.cdf(sigma**2, 1)
chi2_2sigma = scipy.stats.chi2.ppf(ci, dof)
print chi2_1sigma, chi2_2sigma

0.9999999999999881 4.000000000000001


In [120]:
j_bf    = 10**logJbin[np.argmin(chi2bin)]
chi2_bf = chi2bin[np.argmin(chi2bin)]
print ("%.2e" %j_bf)
pos   = np.where(chi2bin <= (chi2_bf + chi2_1sigma))
j_min = 10**np.min(logJbin[pos])
j_max = 10**np.max(logJbin[pos[0]])
print ("Jmin = %.2e and Jmax = %.2e" %(j_min, j_max))

1.25e+22
Jmin = 8.59e+21 and Jmax = 6.65e+22
